In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#Transforming the columns
numerical_transformer = SimpleImputer(strategy = 'constant')
categorical_transformer = Pipeline(steps = [
                                ('imputer', SimpleImputer(strategy = 'most_frequent')),
                                ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
                                ])

#Putting the preproccesing steps together
preprocessor = ColumnTransformer(transformers = [
                                ('num', numerical_transformer, numerical_cols),
                                ('cat', categorical_transformer, categorical_cols)
                                ])

In [5]:
from sklearn.ensemble import RandomForestRegressor

#Creating the model
model = RandomForestRegressor(n_estimators = 60, random_state = 0)

In [6]:
from sklearn.metrics import mean_absolute_error

#Creating the final pipeline
my_pipeline = Pipeline(steps = [
                    ('preprocessor', preprocessor),
                    ('model', model)
                    ])
#Fitting the model
my_pipeline.fit(X_train, y_train)

#Predicting sale prices
preds = my_pipeline.predict(X_valid)

#printing the mean absolute error
print(mean_absolute_error(y_valid, preds))

161463.22574543563
